<a href="https://colab.research.google.com/github/NIcknameInvalido/star_wars/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de bibliotecas

In [ ]:
import requests as req
import pandas as pd
import re as reg
import sqlite3 as slite

# conexão com o banco de dados SqlLite3
---



In [ ]:
conn = slite.connect("db_star_wars")

# Consumo das API'S fornecidas pelo  https://swapi.dev/

In [ ]:
characters = req.get('https://swapi.dev/api/people')
planets = req.get('https://swapi.dev/api/planets')
films = req.get('https://swapi.dev/api/films')

## Criação de dataframes pandas para manipulação de dados

In [17]:
df_charcter = pd.json_normalize(characters.json(), 'results')
df_planets = pd.json_normalize(planets.json(), 'results')
df_films = pd.json_normalize(films.json(), 'results')

# Tratamentos de dados

### Funções

In [18]:
def extractNumberFromUrl(url):
    found = reg.search(r'/(\d+)/', url)
    return int(found.group(1)) if found else None

### Tratamento do dataframe Characters

In [19]:
df_charcter['characterID'] = df_charcter['url'].apply(extractNumberFromUrl)
df_charcter['planetID'] = df_charcter['homeworld'].apply(extractNumberFromUrl)
df_charcter['filmsID'] = df_charcter['films'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_charcter['gender'] = df_charcter['gender'].replace('n/a', '')

### Tratamento do dataframe Planets

In [ ]:
df_planets['planetID'] = df_planets['url'].apply(extractNumberFromUrl)

### Tratamento do dataframe Films

In [20]:
df_films['filmsID'] = df_films['url'].apply(extractNumberFromUrl)
df_films['characters'] = df_films['characters'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['planets'] = df_films['planets'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['vehicles'] = df_films['vehicles'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['starships'] = df_films['starships'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['species'] = df_films['species'].apply(lambda x: [extractNumberFromUrl(url) for url in x])

## Visualização de dados

In [13]:
df_charcter[['characterID', 'name']].head(3)

,characterID,name
0,1,Luke Skywalker
1,2,C-3PO


In [22]:
df_films

,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url,filmsID
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/,1
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[1, 2, 3, 4, 5, 10, 13, 14, 18, 20, 21, 22, 23...","[4, 5, 6, 27]","[3, 10, 11, 12, 15, 17, 21, 22, 23]","[8, 14, 16, 18, 19, 20]","[1, 2, 3, 6, 7]",2014-12-12T11:26:24.656000Z,2014-12-15T13:07:53.386000Z,https://swapi.dev/api/films/2/,2
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"[1, 2, 3, 4, 5, 10, 13, 14, 16, 18, 20, 21, 22...","[1, 5, 7, 8, 9]","[2, 3, 10, 11, 12, 15, 17, 22, 23, 27, 28, 29]","[8, 16, 18, 19, 24, 25, 26, 30]","[1, 2, 3, 5, 6, 8, 9, 10, 15]",2014-12-18T10:39:33.255000Z,2014-12-20T09:48:37.462000Z,https://swapi.dev/api/films/3/,3
3,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"[2, 3, 10, 11, 16, 20, 21, 32, 33, 34, 35, 36,...","[1, 8, 9]","[31, 32, 39, 40, 41]","[33, 34, 35, 36, 37, 38, 42]","[1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, ...",2014-12-19T16:52:55.740000Z,2014-12-20T10:54:07.216000Z,https://swapi.dev/api/films/4/,4
4,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"[2, 3, 6, 7, 10, 11, 20, 21, 22, 33, 35, 36, 4...","[1, 8, 9, 10, 11]","[21, 32, 39, 43, 47, 48, 49, 52, 58]","[4, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57]","[1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, ...",2014-12-20T10:57:57.886000Z,2014-12-20T20:18:48.516000Z,https://swapi.dev/api/films/5/,5
5,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"[1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 20, 21, ...","[1, 2, 5, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19]","[2, 32, 48, 59, 61, 63, 64, 65, 66, 68, 74, 75]","[33, 50, 53, 56, 60, 62, 67, 69, 70, 71, 72, 7...","[1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 2...",2014-12-20T18:49:38.403000Z,2014-12-20T20:47:52.073000Z,https://swapi.dev/api/films/6/,6


In [23]:
df_planets.head(3)

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/


In [29]:
df_charcter_expanded = df_charcter.explode('filmsID')
df_charcter_film = df_charcter_expanded[['characterID', 'filmsID']]
df_charcter_film = pd.merge(df_charcter_film, df_films, how='inner')
df_charcter_film(df_charcter_film, )


,characterID,filmsID,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url
0,1,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
1,2,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
2,3,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
3,4,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
4,5,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/


# **Banco de dados**

---



## Deleção das tabelas para novas inserções

In [ ]:

conn.execute("""
    DROP TABLE IF EXISTS dim_planets
""")
conn.execute("""
    DROP TABLE IF EXISTS fact_films
""")
conn.execute("""
    DROP TABLE IF EXISTS dim_characters
""")
conn.commit()

## Criação das tabelas para armazenar os dados

In [ ]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS planets(
        planetID INTEGER NOT NULL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        rotation_period INTEGER NOT NULL,
        orbital_period INTEGER,
        diameter INTEGER,
        climate VARCHAR(15),
        gravity VARCHAR(30),
        terrain VARCHAR(60),
        surface_water INTEGER
    )
""")


conn.execute("""
    CREATE TABLE IF NOT EXISTS films(
        filmID INTEGER NOT NULL PRIMARY KEY,
        title VARCHAR(255),
        release_date DATE
    )
""")

conn.execute("""
    CREATE TABLE IF NOT EXISTS characters(
        characterID INTEGER NOT NULL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        height INTEGER,
        mass INTEGER,
        hair_color VARCHAR(15),
        skin_color VARCHAR(15),
        eye_color VARCHAR(15),
        birth_year VARCHAR(30),
        gender VARCHAR(15)
    )
""")

conn.commit()

## Inserção de registros

In [ ]:
data_characters = df_charcter.to_dict(orient='records')
data_films = df_films.to_dict(orient='records')
data_planets = df_planets.to_dict(orient='records')

In [ ]:

for films in data_films:
    conn.execute(
        """INSERT INTO films (filmID, title, release_date) VALUES (?, ?, ?) """,
            (
                films['filmsID'],
                films['title'],
                films['release_date'],
            )
    )
    conn.commit()

for planet in data_planets:
    conn.execute(
        """INSERT INTO planets (planetID, name, rotation_period, orbital_period, diameter, climate, gravity, terrain, surface_water) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) """,
            (
                planet['planetID'],
                planet['name'],
                planet['rotation_period'],
                planet['orbital_period'],
                planet['diameter'],
                planet['climate'],
                planet['gravity'],
                planet['terrain'],
                planet['surface_water']
            )
    )
    conn.commit()




for character in data_characters:
    conn.execute(
        """INSERT INTO characters (characterID, name, height, mass, hair_color, skin_color, eye_color, birth_year, gender) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) """,
            (
                character['characterID'],
                character['name'],
                character['height'],
                character['mass'],
                character['hair_color'],
                character['skin_color'],
                character['eye_color'],
                character['birth_year'],
                character['gender']
            )
    )
    conn.commit()